In [1]:
import ee
import geemap
import geopandas as gpd
import geersd

from pph import core
from pph import ft
from pph import datautils

In [2]:
ee.Initialize()

# Fetch Landsat 8 SR for Fourier Transform

In [3]:
aoi = datautils.load_moose_mount_aoi()
dependent = 'SR_B5'
time_series = (
    geersd.Landsat8SR()
    .filterBounds(aoi)
    .filterDate('2017', '2024')
    .filter(ee.Filter.eq('WRS_ROW', 25))
    .filter(ee.Filter.eq('WRS_PATH', 34))
    .applyScalingFactor()
    .applyCloudMask()
    .select(dependent)
)

In [4]:
from geeft import compute

ft = compute(time_series, dependent=dependent, modes=2)

In [5]:
Map = geemap.Map()

Map.addLayer(ft.clip(aoi))
Map.addLayer(time_series)

Map.centerObject(aoi, 10)

Map

Map(center=[49.76291503423149, -102.41828841732854], controls=(WidgetControl(options=['position', 'transparent…

In [7]:
ft_task = datautils.ee_l8_image_to_gdrive(
    image=ft,
    region=aoi,
    folder='moose-mountain-ft-sr-b5',
    filename='moose-mountain-ft-sr-b5'
)

ft_task.start()